# Exam 2, Fall 2021 (100 pts total)

files needed = ('pl.csv', 'nval.csv', 'wdi_q4.csv') which can be found in `exam2_f21s1_data.zip`

You have 75 minutes to complete this exam.

Answer all the questions below in this notebook. You should insert as many cells into the notebook as you need. When you are finished, upload your finished notebook to Canvas. 

* You may use your notes and the Internet, but you cannot work with others.  
* Import any packages you need to complete this exam.
* Do not modify the data files. All data manipulation should happen in your code.

**A word on plotting:** At this point in the course, I expect you to create graphically excellent visualizations. 

Remember, jupyter notebooks and python have lots of built in help facilities. 

## Question 0 (5 points): Yuan, Meng
 
Replace 'Last, First' above with your actual name. Enter it as: last name, first name.

## Question 1 (15 points): Time series/APIs 

1. From FRED, retrieve the daily price of oil (DCOILWTICO).  
2. Compute the average price of oil in 2020 and 2021. Print them out as: 

   "The mean price per barrel of oil in 2020 is \\$xx.xx and in 2021 is $xx.xx."

In [1]:
import pandas as pd               
import matplotlib.pyplot as plt   
import datetime as dt  
import numpy as np
import pandas_datareader.data as web
import seaborn as sns
import matplotlib.dates as mdates

In [2]:
#q1
#1
start = dt.datetime(2020,1,1) 
codes = 'DCOILWTICO'
fred = web.DataReader(codes, 'fred', start)
price20 = fred.loc['2020-01-01':'2020-12-31'].mean()
price21 = fred.loc['2021-01-01':'2021-12-31'].mean()
#2
print("The mean price per barrel of oil in 2020 is ${0:.2f} and in 2021 is ${1:.2f}.".format(price20[0], price21[0]))


The mean price per barrel of oil in 2020 is $39.16 and in 2021 is $67.46.


## Question 2A (15 points): Merging 

1. Load the files 'pl.csv' and 'nval.csv'. They contain the price index and nominal value added by U.S. industries for 2010&ndash;2020.
2. Merge the two DataFrames together. 
3. In the merged DataFrame, compute real value added by industry:

$$\text{real value added} = \frac{\text{nominal value added}}{\text{price index/100}}.$$

4. Print out the top 2 rows of your DataFrame.

In [3]:
#q2a
#1
pl = pd.read_csv('pl.csv')
nval = pd.read_csv('nval.csv')

print(pl.head(2))
print(nval.head(2))

print(pl.shape)
print(nval.shape)

                              ind  year       p
0          Gross domestic product  2010  96.164
1          Gross domestic product  2011  98.157
                              ind  year      nva
0          Gross domestic product  2010  15049.0
1          Gross domestic product  2011  15599.7
(1067, 3)
(1067, 3)


In [4]:
#2
merge_1 = pd.merge(left=pl, right=nval, on=['ind','year'], how='inner')

In [5]:
#3
merge_1['rv add'] = merge_1['nva'] / (merge_1['p'] / 100)
#4
merge_1.head(2)

,ind,year,p,nva,rv add
0,Gross domestic product,2010,96.164,15049.0,15649.307433
1,Gross domestic product,2011,98.157,15599.7,15892.600630


## Question 2B (15 points): Indexing

If you completed question 2A, use your results in this question. If not, use the file 'q2a_f21.csv' to complete this question.

1. Report real value added for "Utilities" in 2015. 
2. Print it out as: 

   "Real value added in utilities is xxx.xx billion dollars."

In [6]:
#q2b

rvadd = pd.read_csv('q2a_f21.csv', parse_dates=['year']) 
#1
rvadd.set_index('year', inplace = True)

Utilities = rvadd.loc['2015']['rva'].sum()
print("Real value added in utilities is {0:.2f} billion dollars.".format(Utilities))

Real value added in utilities is 81154.18 billion dollars.


## Question 3 (15 points): Groupby/Plotting
If you completed question 2A, use your results in this question. If not, use the file 'q2a_f21.csv' to complete this question.

1. Compute the average real value added and the median real value added for each industry, 2010&ndash;2020.
2. Drop 'Gross domestic product' and 'Private industries'. (If you have trouble dropping these industries, skip this step and go to step 3.)
3. Create a scatter plot with a regression line. Average real value added by industry is the y-axis and median real value added by industry is the x-axis.
    1. Use black for the scatter points and the line
    1. Do not plot a confindence interval

In [7]:
#q3
q3 = pd.read_csv('q2a_f21.csv')
q3.head(3)
#1
agg_value = q3[['rva','ind']].groupby('ind').agg(['mean','median'])

agg_value

rva              
                                                         mean        median
ind                                                                        
                Housing                           1565.360560   1555.353405
                Other real estate                  417.564415    428.690427
            Accommodation                          133.072005    134.721660
            Administrative and support services    465.913618    463.527552
            Ambulatory health care services        644.178656    641.519577
...                                                       ...           ...
    Transportation and warehousing                 504.558929    494.351619
    Utilities                                      278.797050    280.700000
    Wholesale trade                               1053.382182   1083.882554
Government                                        2173.361922   2175.829770
Private industries                               15114.010288  15197.892720

[95 rows x 2 columns]

In [8]:
#2
agg_value = agg_value.drop(['Gross domestic product','Private industries'])
agg_value

KeyError: "['Gross domestic product'] not found in axis"

Create a scatter plot with a regression line. Average real value added by industry is the y-axis and median real value added by industry is the x-axis.
Use black for the scatter points and the line
Do not plot a confindence interval

In [ ]:
agg_value.columns = agg_value.columns.droplevel(level = 0)

In [ ]:
#3

fig, ax = plt.subplots(figsize=(15,6))
sns.regplot(x='median',                                       
            y='mean',                                        
            data=agg_value,                                          
            ax = ax,                                         
            color = 'black',                                  
            ci = 0)  

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_title('Median and average real value')
ax.set_xlabel('Median real value')
ax.set_ylabel('Average real value')
plt.show()

## Question 4 (15 points): Reshaping 

The file 'wdi_q4.csv' contains data on real GDP and the working age population (15-64 years) for China and Japan from 1970 to 2019. 

1. Compute the ratio of GDP to the working age population for both countries for all years. Name the variable 'gdpc'.
2. Arrange your DataFrame so that the index is the country (outermost index) and date (innermost index) and the columns are the variable names. Sort both of the indexes. 
3. Print out the first two rows of your DataFrame.

In [ ]:
#q4

wdi = pd.read_csv('wdi_q4.csv',parse_dates=['DATE'])
#1
wdi.set_index(['DATE', 'var'], inplace=True)
wdi = wdi.sort_index(axis=0)
wdi2 = wdi.transpose() 
wdi3 = wdi2.stack('DATE')
wdi3['gpdc'] = wdi3['gdp'] / wdi3['pop']

wdi3.head(2)

In [ ]:
#2
wdi3 = wdi3.sort_index(axis=0)
#3
wdi3.head(2)

## Question 5 (15 points): Transformations

Create the `dogs` DataFrame and use pandas funtions to:

1. Remove the '\n' characters from the dogs' names.
3. Create a new column in `dogs` named `year` that holds the year in which each dog was born.
4. In the column `sex`, replace 0 with the text 'good boy' and 1 with the text 'good girl'.
5. Print out `dogs` 

```python
dogs = pd.DataFrame({'name':['Buster\n', 'Subee\n', 'Jax', 'Izzy'], 
                     'dob':['2005-05-06', '2005-10-07', '2019-07-15', '2020-12-22'],
                    'sex':[0,1,0,1]})
```

In [ ]:
#q5
dogs = pd.DataFrame({'name':['Buster\n', 'Subee\n', 'Jax', 'Izzy'], 
                     'dob':['2005-05-06', '2005-10-07', '2019-07-15', '2020-12-22'],
                    'sex':[0,1,0,1]})

#1
dogs['name'] = dogs['name'].str.strip()
dogs

In [ ]:
#2
dogs['dob'] = pd.to_datetime(dogs['dob'])

In [ ]:
dogs['year'] =dogs['dob'].dt.year

dogs

In [ ]:
#3
dogs['sex'] = dogs['sex'].replace({0:'good boy',1: 'good girl'})

dogs

You are finished! Upload your completed notebook to Canvas. 

Enjoy the break, and I will see you next Monday.